### Association rules by Top 5 popular Departments 
#### Department - Dairy Eggs

In [2]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [3]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [4]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [5]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [6]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [ ]:
combine_train_prior

In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="dairy eggs"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [10]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [11]:
grouped_data

,order_id,product_name
6,98,Unsalted Cultured Butter
7,98,Whole Milk Greek Blended Vanilla Bean Yogurt
8,98,Queso Fresco
9,98,Organic Coconut Milk
10,98,Organic Unsweetened Almond Milk
11,98,Organic Chocolate Almondmilk Pudding
12,98,Organic Whole Grassmilk Milk
13,98,Organic 2% Buttermilk
14,98,Organic Stringles Mozzarella String Cheese
15,98,Baby Swiss Slices Cheese


In [12]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [ ]:
table1

In [14]:
#relace NAN with 0
table1=table1.fillna(0)

In [15]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1
basket_sets = table1.applymap(encode_units)

In [16]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [17]:
frequent_itemsets

,support,itemsets
0,0.012658,[1% Lowfat Milk]
1,0.012258,[2% Reduced Fat DHA Omega-3 Reduced Fat Milk]
2,0.024360,[2% Reduced Fat Milk]
3,0.014971,[2% Reduced Fat Organic Milk]
4,0.014340,[Blueberry Yoghurt]
5,0.010401,[Blueberry on the Bottom Nonfat Greek Yogurt]
6,0.010835,[Cherry Pomegranate Greek Yogurt]
7,0.015410,[Colby Cheese Sticks]
8,0.013730,[Cream Cheese Spread]
9,0.011198,[Fat Free Blueberry Yogurt]


In [18]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

In [19]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Icelandic Style Skyr Blueberry Non-fat Yogurt),(Vanilla Skyr Nonfat Yogurt),0.029076,0.025750,0.012142,0.417576,16.216821,0.011393,1.672752
1,(Vanilla Skyr Nonfat Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.025750,0.029076,0.012142,0.471526,16.216821,0.011393,1.837222
2,(Total 2% with Strawberry Lowfat Greek Straine...,(Total 2% All Natural Greek Strained Yogurt wi...,0.043433,0.031430,0.010926,0.251564,8.003830,0.009561,1.294125
3,(Total 2% All Natural Greek Strained Yogurt wi...,(Total 2% with Strawberry Lowfat Greek Straine...,0.031430,0.043433,0.010926,0.347631,8.003830,0.009561,1.466297
4,(Total 2% Lowfat Greek Strained Yogurt with Pe...,(Total 2% with Strawberry Lowfat Greek Straine...,0.031283,0.043433,0.014900,0.476298,10.966245,0.013541,1.826548
5,(Total 2% with Strawberry Lowfat Greek Straine...,(Total 2% Lowfat Greek Strained Yogurt with Pe...,0.043433,0.031283,0.014900,0.343059,10.966245,0.013541,1.474587
6,(Non Fat Raspberry Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.024847,0.029076,0.013137,0.528715,18.183646,0.012415,2.060160
7,(Icelandic Style Skyr Blueberry Non-fat Yogurt),(Non Fat Raspberry Yogurt),0.029076,0.024847,0.013137,0.451815,18.183646,0.012415,1.778876
8,(Organic Whole Milk),(Organic Whole String Cheese),0.114962,0.069718,0.013838,0.120372,1.726567,0.005823,1.057586
9,(Organic Whole String Cheese),(Organic Whole Milk),0.069718,0.114962,0.013838,0.198489,1.726567,0.005823,1.104212


In [20]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Icelandic Style Skyr Blueberry Non-fat Yogurt),(Vanilla Skyr Nonfat Yogurt),0.029076,0.025750,0.012142,0.417576,16.216821,0.011393,1.672752
1,(Vanilla Skyr Nonfat Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.025750,0.029076,0.012142,0.471526,16.216821,0.011393,1.837222
2,(Total 2% with Strawberry Lowfat Greek Straine...,(Total 2% All Natural Greek Strained Yogurt wi...,0.043433,0.031430,0.010926,0.251564,8.003830,0.009561,1.294125
3,(Total 2% All Natural Greek Strained Yogurt wi...,(Total 2% with Strawberry Lowfat Greek Straine...,0.031430,0.043433,0.010926,0.347631,8.003830,0.009561,1.466297
4,(Total 2% Lowfat Greek Strained Yogurt with Pe...,(Total 2% with Strawberry Lowfat Greek Straine...,0.031283,0.043433,0.014900,0.476298,10.966245,0.013541,1.826548
5,(Total 2% with Strawberry Lowfat Greek Straine...,(Total 2% Lowfat Greek Strained Yogurt with Pe...,0.043433,0.031283,0.014900,0.343059,10.966245,0.013541,1.474587
6,(Non Fat Raspberry Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.024847,0.029076,0.013137,0.528715,18.183646,0.012415,2.060160
7,(Icelandic Style Skyr Blueberry Non-fat Yogurt),(Non Fat Raspberry Yogurt),0.029076,0.024847,0.013137,0.451815,18.183646,0.012415,1.778876
8,(Organic Whole Milk),(Organic Whole String Cheese),0.114962,0.069718,0.013838,0.120372,1.726567,0.005823,1.057586
9,(Organic Whole String Cheese),(Organic Whole Milk),0.069718,0.114962,0.013838,0.198489,1.726567,0.005823,1.104212


In [21]:
final_result.to_csv("Department_DairyEggsApriori.csv",sep=',')

In [22]:
test1=pd.read_csv("Department_DairyEggsApriori.csv")

In [23]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [24]:
test1=test1.replace({"u'": ''}, regex=True)

In [25]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [26]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [27]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [28]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Icelandic Style Skyr Blueberry Non-fat Yogurt,Vanilla Skyr Nonfat Yogurt,0.029076,0.025750,0.012142,0.417576,16.216821,0.011393,1.672752
1,Vanilla Skyr Nonfat Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,0.025750,0.029076,0.012142,0.471526,16.216821,0.011393,1.837222
2,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% All Natural Greek Strained Yogurt wit...,0.043433,0.031430,0.010926,0.251564,8.003830,0.009561,1.294125
3,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% with Strawberry Lowfat Greek Strained...,0.031430,0.043433,0.010926,0.347631,8.003830,0.009561,1.466297
4,Total 2% Lowfat Greek Strained Yogurt with Peach,Total 2% with Strawberry Lowfat Greek Strained...,0.031283,0.043433,0.014900,0.476298,10.966245,0.013541,1.826548
5,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% Lowfat Greek Strained Yogurt with Peach,0.043433,0.031283,0.014900,0.343059,10.966245,0.013541,1.474587
6,Non Fat Raspberry Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,0.024847,0.029076,0.013137,0.528715,18.183646,0.012415,2.060160
7,Icelandic Style Skyr Blueberry Non-fat Yogurt,Non Fat Raspberry Yogurt,0.029076,0.024847,0.013137,0.451815,18.183646,0.012415,1.778876
8,Organic Whole Milk,Organic Whole String Cheese,0.114962,0.069718,0.013838,0.120372,1.726567,0.005823,1.057586
9,Organic Whole String Cheese,Organic Whole Milk,0.069718,0.114962,0.013838,0.198489,1.726567,0.005823,1.104212


In [29]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_DairyEggsApriori.csv")

In [30]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Icelandic Style Skyr Blueberry Non-fat Yogurt,Vanilla Skyr Nonfat Yogurt,0.029076,0.025750,0.012142,0.417576,16.216821,0.011393,1.672752
1,Vanilla Skyr Nonfat Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,0.025750,0.029076,0.012142,0.471526,16.216821,0.011393,1.837222
2,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% All Natural Greek Strained Yogurt wit...,0.043433,0.031430,0.010926,0.251564,8.003830,0.009561,1.294125
3,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% with Strawberry Lowfat Greek Strained...,0.031430,0.043433,0.010926,0.347631,8.003830,0.009561,1.466297
4,Total 2% Lowfat Greek Strained Yogurt with Peach,Total 2% with Strawberry Lowfat Greek Strained...,0.031283,0.043433,0.014900,0.476298,10.966245,0.013541,1.826548
5,Total 2% with Strawberry Lowfat Greek Strained...,Total 2% Lowfat Greek Strained Yogurt with Peach,0.043433,0.031283,0.014900,0.343059,10.966245,0.013541,1.474587
6,Non Fat Raspberry Yogurt,Icelandic Style Skyr Blueberry Non-fat Yogurt,0.024847,0.029076,0.013137,0.528715,18.183646,0.012415,2.060160
7,Icelandic Style Skyr Blueberry Non-fat Yogurt,Non Fat Raspberry Yogurt,0.029076,0.024847,0.013137,0.451815,18.183646,0.012415,1.778876
8,Organic Whole Milk,Organic Whole String Cheese,0.114962,0.069718,0.013838,0.120372,1.726567,0.005823,1.057586
9,Organic Whole String Cheese,Organic Whole Milk,0.069718,0.114962,0.013838,0.198489,1.726567,0.005823,1.104212


In [196]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()